In [4]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [5]:
api_key = input("Please enter your YouTube Data API key: ")
channel_ids=['UCX6OQ3DkcsbYNE6H8uQQuVA',
             'UCChmJrVa8kDg05JfCmxpLRw',
             'UCkX0NXCP9kCLtk30kjTXIWg',
             'UCXqJrBG563pZnaqLoxHjUDg',
             'UCHoLIMtg_OigNlJmjHX9J8Q',
             'UCMOLiZpKXp-w5CNfqJObQUw',
             'UCtAXJ4DNpshfVjLIMq9pIRw']

youtube= build('youtube','v3',developerKey=api_key)

In [6]:
def get_channel(youtube,channel_ids):
    
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids)
                )
    response = request.execute() 
    

    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)




In [7]:
channel_stats=get_channel(youtube,channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Something_beyond,18,8533,46,UUkX0NXCP9kCLtk30kjTXIWg
1,Darshil Parmar,127000,5849705,149,UUChmJrVa8kDg05JfCmxpLRw
2,Rohit & Kanupriya,2230000,2114462647,321,UUtAXJ4DNpshfVjLIMq9pIRw
3,MrBeast,255000000,47881772057,790,UUX6OQ3DkcsbYNE6H8uQQuVA
4,KALEO,1940000,1538340396,68,UUXqJrBG563pZnaqLoxHjUDg
5,coder2j,9770,904346,48,UUHoLIMtg_OigNlJmjHX9J8Q
6,KEEMOKAZI,9660000,10494733877,1316,UUMOLiZpKXp-w5CNfqJObQUw


In [8]:
playlistid=channel_stats['playlistId']
playlistid

0    UUkX0NXCP9kCLtk30kjTXIWg
1    UUChmJrVa8kDg05JfCmxpLRw
2    UUtAXJ4DNpshfVjLIMq9pIRw
3    UUX6OQ3DkcsbYNE6H8uQQuVA
4    UUXqJrBG563pZnaqLoxHjUDg
5    UUHoLIMtg_OigNlJmjHX9J8Q
6    UUMOLiZpKXp-w5CNfqJObQUw
Name: playlistId, dtype: object

In [9]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids


In [10]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    

len(Video_ids)

2739

In [11]:
def get_video_details(youtube, video_ids):    
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [12]:
videoDetails=get_video_details(youtube, Video_ids)
videoDetails

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,YZSv1Q8zuWo,Something_beyond,Elon Musk's Space X With Kaleo-Way-Down-We-Go,Join us as we witness the incredible journey o...,"[spacex, space x, spacex night launch, spacex ...",2024-04-27T07:45:27Z,8,3,None,0,PT3M38S,hd,false
1,njzjxMftEe8,Something_beyond,Catching super heavy,,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-04-07T15:51:26Z,2,1,None,0,PT1M,hd,false
2,hmxfn4cTdlQ,Something_beyond,Watch Falcon 9 launch the Eutelsat Group EUTEL...,Witness the spectacular Falcon 9 launch as it ...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-31T15:02:09Z,3,2,None,0,PT1M,hd,false
3,iXVGrwRxV2c,Something_beyond,Full duration static fire of all six Raptor en...,Watch this incredible footage of the full dura...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-27T12:36:50Z,5,2,None,0,PT15S,hd,false
4,vW_LAxHJY_w,Something_beyond,Falcon 9 launches Starlink satellites to orbit...,Watch the latest Falcon 9 launch as it sends a...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-27T12:32:30Z,438,11,None,0,PT47S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,KyrXPBGT80w,KEEMOKAZI,Keemokazi - Flexin,“Flexin” is Keemokazi’s newest release. The si...,None,2020-04-10T04:02:48Z,156684,4211,None,226,PT3M16S,hd,false
2735,kSRUiinXLYs,KEEMOKAZI,Q&A with Keemokazi!,Get to know me with my latest video! \nThanks ...,None,2020-04-09T01:59:02Z,289370,13718,None,929,PT6M25S,hd,false
2736,yQo0noQwsKQ,KEEMOKAZI,Top 10 Keemokazi pranks!,Thank you guys for all the support!\n\nInstagr...,None,2020-04-07T02:37:43Z,2076699,48792,None,0,PT8M23S,hd,false
2737,7VTbryaAFYU,KEEMOKAZI,Keemokazi Tik Tok Compilation,Thank you guys for all the support! If you us...,None,2020-03-28T02:26:52Z,1113627,21651,None,950,PT18M9S,hd,false


In [13]:
videoDetails.to_csv('Data\\apiData.csv', index=False)
